# Intro

I'm trying to learn the very basics with this exercise. My goal is to train a linear regression model with a subset of columns from this interesting dataset in order to predict the value of a used car.

Any help or advice is welcome!!!

### Changelist

* added name length feature

* better study on the data
* used seaborn to plot
* added random forest and xgboost algorithms

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model, preprocessing, svm
from sklearn.preprocessing import StandardScaler, Normalizer
import math
import matplotlib
import seaborn as sns


# Preparing data

## Reading from file

Just reading the file and printing some lines.

In [ ]:
df = pd.read_csv('autos.csv.gz', sep=',', header=0, compression='gzip',encoding='cp1252')
df.head()

## Cleaning data from outliers and dirty values

Cleaning data from duplicates, NaNs and selecting reasonable ranges for columns


In [ ]:
print("Too new: %d" % df.loc[df.yearOfRegistration >= 2017].count()['name'])
print("Too old: %d" % df.loc[df.yearOfRegistration < 1950].count()['name'])
print("Too cheap: %d" % df.loc[df.price < 100].count()['name'])
print("Too expensive: " , df.loc[df.price > 150000].count()['name'])
print("Too few km: " , df.loc[df.kilometer < 5000].count()['name'])
print("Too many km: " , df.loc[df.kilometer > 200000].count()['name'])
print("Too few PS: " , df.loc[df.powerPS < 10].count()['name'])
print("Too many PS: " , df.loc[df.powerPS > 500].count()['name'])
print("Fuel types: " , df['fuelType'].unique())
print("Offer types: " , df['offerType'].unique())
print("Sellers: " , df['seller'].unique())
print("Damages: " , df['notRepairedDamage'].unique())
print("Pics: " , df['nrOfPictures'].unique()) # nrOfPictures : number of pictures in the ad (unfortunately this field contains everywhere a 0 and is thus useless (bug in crawler!) )
print("Postale codes: " , df['postalCode'].unique())
print("Vehicle types: " , df['vehicleType'].unique())
print("Brands: " , df['brand'].unique())

#print("tourans: " , df[df['model']=='touran'][['name','vehicleType','powerPS','yearOfRegistration']])

# Cleaning data
valid_models = df.dropna()

#### Removing the duplicates
dedups = valid_models.drop_duplicates(['name','seller','offerType','price','abtest','vehicleType','yearOfRegistration'
                         ,'gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType'
                         ,'notRepairedDamage','postalCode'])

#### Removing the outliers
no_outliers = dedups[
        (valid_models.yearOfRegistration <= 2016) 
      & (valid_models.yearOfRegistration >= 1950) 
      & (valid_models.price >= 100) 
      & (valid_models.price <= 150000) 
      & (valid_models.powerPS >= 10) 
      & (valid_models.powerPS <= 500)]

print("\nData kept for analisys: %d percent of the entire set" % (100 * no_outliers['name'].count() / df['name'].count()))


## Studying correlations between some features and sell price

I want to see how data is distributed and how it's related to the sell price. I start considering the numeric fields already present in the dataset, then I'll do other studies after having encoded the string fields.

In [ ]:
p = no_outliers
ax = sns.jointplot(x='powerPS', y='price',data=p[['powerPS','price']], alpha=0.1, size=8)
plt.title('Price distribution for car power')

plt.xlim(0, 501)
plt.ylim(0, 160000)


Price at varying yearOfRegistration

In [ ]:
p = no_outliers
#plt.scatter(p['yearOfRegistration'], p['price'], alpha=.1, marker='o', color='b')
ax = sns.jointplot(x='yearOfRegistration', 
                   y='price',
                   data=p[['yearOfRegistration','price']], 
                   alpha=0.1, size=8)

plt.title('Price distribution for year')

plt.ylim(0, 160000)



Price at varying kilometers

In [ ]:
p = no_outliers
#plt.scatter(p['kilometer'], p['price'], alpha=.1, marker='o', color='b')
ax = sns.jointplot(x='kilometer', y='price',data=p[['kilometer','price']], alpha=0.1, size=8)

plt.title('Price distribution for kilometers')
#plt.xlabel('Kilometers')
#plt.ylabel('Price (logarithmic)')
#plt.yscale('log')
plt.ylim(0, 160000)
plt.xticks([5000,10000,20000,30000,40000,50000,60000,70000,80000,90000,100000,125000,150000], rotation='vertical')


### Features engineering

In [ ]:
p = no_outliers
p['namelen'] = [len(n) for n in p['name']]

print(p['namelen'].head())
#plt.scatter(p['namelen'][p['model']=='golf'], p['price'][p['model']=='golf'], alpha=.1, marker='o', color='b')
ax = sns.jointplot(x='namelen', y='price',data=p[['namelen','price']][p['model']=='golf'], alpha=0.1, size=8)

p = p[p.namelen < 100] # removed the 22k chars name

plt.title('Price distribution for name length')
plt.ylim(0, 45000)
plt.xlim(0, 70)


Here I plotted the name length vs car price for all the vw golfs and it seems that more complete names bring to a bit higher sell price. Another explanation could be that a longer name includes more optionals and accessories and therefore the price is obviously higher.

Very short and very long names do not work well.

## Selecting only relevant columns

Here I select the columns that I think are useful for determining a car's price.


In [ ]:

# only relevant columns
rel_cols = no_outliers[['price'
                        ,'yearOfRegistration'
                        ,'gearbox'
                        ,'powerPS'
                        ,'model'
                        ,'kilometer'
                        ,'fuelType'
                        ,'vehicleType'
                        ,'monthOfRegistration'
                        ,'brand'
                        ,'notRepairedDamage'
                       , 'namelen']]
rel_cols.head()

## Encoding

This part encodes the string fields to numerical values, in order to study the rest of the columns and to make some regressions.

In [ ]:
labels = ['gearbox', 'notRepairedDamage', 'model', 'brand', 'fuelType', 'vehicleType']
les = {}

for l in labels:
    les[l] = preprocessing.LabelEncoder()
    les[l].fit(rel_cols[l])
    #print(les[l].classes_)
    tr = les[l].transform(rel_cols[l]) 
    rel_cols.loc[:, l + '_feat'] = pd.Series(tr, index=rel_cols.index)

df_autos = rel_cols[ ['price'
                        ,'yearOfRegistration'
                        ,'powerPS'
                        ,'kilometer'
                        ,'monthOfRegistration'
                        , 'namelen'] 
                    + [x+"_feat" for x in labels]]

df_autos['yearOfRegistration'] = df_autos['yearOfRegistration'].apply(lambda x: x-2000)

#print(df_autos['yearOfRegistration'])
autos = df_autos.values.astype(float)

Y = autos[:,0]
X = autos[:,1:]


### Studying the other correlations

In [ ]:
plt.title('Pearson Correlation')
sns.heatmap(df_autos.astype(float).corr(),
            linewidths=0.1,
            vmax=1.0, 
            square=True, 
            cmap="PuBuGn", 
            linecolor='w', 
            annot=False)


In [ ]:
p = df_autos
#plt.scatter(p['powerPS'], p['price'], alpha=.1, marker='o', color='b')
ax = sns.jointplot(x='brand_feat', y='price',data=p[['brand_feat','price']], alpha=0.1, size=8)
plt.title('Price distribution for brands')
#plt.xlabel('Power in HP')
#plt.ylabel('Price (logarithmic)')
#plt.yscale('log')

brands = zip(range(0, len(les['brand'].classes_)), les['brand'].classes_)

s = ''
for b in brands:
    s = s + ' ' + str(b)

print("brands: " + s)

plt.xlim(0, 41)
plt.ylim(0, 160000)

In [ ]:
p = df_autos[df_autos['brand_feat']==37]
#plt.scatter(p['powerPS'], p['price'], alpha=.1, marker='o', color='b')
ax = sns.jointplot(x='model_feat', y='price',data=p[['model_feat','price']], alpha=0.1, size=8)
plt.title('Price distribution for VW models')
#plt.xlabel('Power in HP')
#plt.ylabel('Price (logarithmic)')
#plt.yscale('log')

model = zip(range(0, len(les['model'].classes_)), les['model'].classes_)

s = ''
for b in model:
    s = s + ' ' + str(b)

print("models: " + s)

#plt.xlim(0, 41)
plt.ylim(0, 160000)

In [ ]:
p = df_autos
f = plt.figure()

JG1 = sns.jointplot(x='fuelType_feat', y='price',data=p[['fuelType_feat','price']], alpha=0.1, size=4)
JG2 = sns.jointplot(x='vehicleType_feat', y='price',data=p[['vehicleType_feat','price']], alpha=0.1, size=4)
JG3 = sns.jointplot(x='gearbox_feat', y='price',data=p[['gearbox_feat','price']], alpha=0.1, size=4)
JG4 = sns.jointplot(x='notRepairedDamage_feat', y='price',data=p[['notRepairedDamage_feat','price']], alpha=0.1, size=4)

fuelTypes = zip(range(0, len(les['fuelType'].classes_)), les['fuelType'].classes_)
s = ''
for b in fuelTypes:
    s = s + ' ' + str(b)
print("Fuels: " + s)

vehicleTypes = zip(range(0, len(les['vehicleType'].classes_)), les['vehicleType'].classes_)
s = ''
for b in vehicleTypes:
    s = s + ' ' + str(b)
print("vehicleType: " + s)

gearboxes = zip(range(0, len(les['gearbox'].classes_)), les['gearbox'].classes_)
s = ''
for b in gearboxes:
    s = s + ' ' + str(b)
print("gearbox: " + s)

fuelTypes = zip(range(0, len(les['notRepairedDamage'].classes_)), les['notRepairedDamage'].classes_)
s = ''
for b in fuelTypes:
    s = s + ' ' + str(b)
print("notRepairedDamage: " + s)

#plt.xlim(0, 41)
plt.ylim(0, 160000)

## Correction

Transforming the right-skewed sale price column.


In [ ]:
matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)
prices = pd.DataFrame({"1. Before":Y, "2. After":np.log1p(Y)})
prices.hist()

Y = np.log1p(Y)

# Playing with different models

Trying with some model from scikit learn: LinearRegression, LR with L2 regularization and others.

In [ ]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, Lasso, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score, train_test_split


def cv_rmse(model, x, y):
    r = np.sqrt(-cross_val_score(model, x, y, scoring="neg_mean_squared_error", cv = 5))
    return r

r = range(2003, 2017)
km_year = 10000



## Linear Regression

In [ ]:
linear = linear_model.LinearRegression()

# Percent of the X array to use as training set. This implies that the rest will be test set
test_size = .2

#Split into train and validation
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state = 3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
linear.fit(X_train, y_train)

# Explained variance score: 1 is perfect prediction
score = linear.score(X_test, y_test)
print('Variance score: %.2f' % score)

print("rmse on validation set", cv_rmse(linear, X_test, y_test).mean())

# Last version results:
# Variance score: 0.68
# rmse on validation set 0.639555427197


#### Predicting samples

In [ ]:

def calc_values(model, year, brand, car_model, vehicle, gearbox, fuel, powerPS, km, not_repaired_damage):
    sample = [year
          , powerPS
          , km
          , 1 # month
          , 35 # namelen
          , les['gearbox'].transform([gearbox])
          , les['notRepairedDamage'].transform([not_repaired_damage])
          , les['model'].transform([car_model])
          , les['brand'].transform([brand])
          , les['fuelType'].transform([fuel])
          , les['vehicleType'].transform([vehicle])]
    s_predict = np.expm1(model.predict([sample]))
    return s_predict

# [nan 'coupe' 'suv' 'kleinwagen' 'limousine' 'cabrio' 'bus' 'kombi' 'andere']
vw1 = calc_values(linear, 2016, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 105, 10000, 'nein')
bmw  = calc_values(linear, 2016, 'bmw', '3er', 'limousine', 'manuell', 'diesel', 150, 10000, 'nein')
fiat  = calc_values(linear, 2016, 'fiat', 'punto', 'kleinwagen', 'manuell', 'benzin', 60, 10000, 'nein')
vw2 = calc_values(linear, 2016, 'volkswagen', 'tiguan', 'suv', 'manuell', 'diesel', 130, 10000, 'nein')

print(vw1)
print(bmw)
print(fiat)
print(vw2)

values_linear  = [calc_values(linear, y, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 120, km_year * (2017-y), 'nein') for y in r]
values_linear2  = [calc_values(linear, y, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 120, km_year * 2 * (2017-y), 'nein') for y in r]
values_linear3  = [calc_values(linear, y, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 120, km_year * 3 * (2017-y), 'nein') for y in r]
plt.plot(r, values_linear, label='linear')
plt.plot(r, values_linear2, label='linear 20000km_x_y')
plt.plot(r, values_linear3, label='linear 30000km_x_y')

#plt.grid()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


## L1 regularized LR


In [ ]:

alphas = [0.005,0.05, 0.1, 0.3, 0.5, 0.7, 1, 2, 3, 5, 10, 20, 25, 30, 50]
cv_lasso = [cv_rmse(Lasso(alpha = alpha), X_train, y_train).mean() for alpha in alphas]

cv_lasso = pd.Series(cv_lasso, index = alphas)
cv_lasso.plot(title = "Validation", figsize=(5,5))
plt.xlabel("alpha")
plt.ylabel("rmse")

model_lasso = Lasso(alpha = cv_lasso.min()).fit(X_train, y_train)

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % model_lasso.score(X_test, y_test))

coef = pd.Series(model_lasso.coef_, index = df_autos.iloc[:,1:].columns)

imp_coef = pd.concat([coef.sort_values()])

print(imp_coef)

print(np.sqrt(np.mean((y_test - model_lasso.predict(X_test))**2)))
print("rmse on validation set", cv_rmse(model_lasso, X_test, y_test).mean())

# Last version results:
# 0.707839916347
# rmse on validation set 0.708803418524


#### Predicting samples

In [ ]:
vw1 = calc_values(model_lasso, 2016, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 105, 10000, 'nein')
bmw  = calc_values(model_lasso, 2016, 'bmw', '3er', 'limousine', 'manuell', 'diesel', 150, 10000, 'nein')
fiat  = calc_values(model_lasso, 2016, 'fiat', 'punto', 'kleinwagen', 'manuell', 'benzin', 60, 10000, 'nein')
vw2 = calc_values(model_lasso, 2016, 'volkswagen', 'tiguan', 'suv', 'manuell', 'diesel', 130, 10000, 'nein')

print(vw1)
print(bmw)
print(fiat)
print(vw2)

values_lasso  = [calc_values(model_lasso, y, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 120, km_year * (2017-y), 'nein') for y in r]
values_lasso2  = [calc_values(model_lasso, y, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 120, km_year * 2 * (2017-y), 'nein') for y in r]
values_lasso3  = [calc_values(model_lasso, y, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 120, km_year * 3 * (2017-y), 'nein') for y in r]
plt.plot(r, values_lasso, label='lasso')
plt.plot(r, values_lasso2, label='lasso 20000km_x_y')
plt.plot(r, values_lasso3, label='lasso 30000km_x_y')

#plt.grid()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


## L2 regularized LR

In [ ]:

alphas = [-10, -5, -2, -1, 0, 0.005,0.05, 0.1, 0.3, 1, 3, 5, 10, 20, 25, 30, 50]
cv_ridge = [cv_rmse(Ridge(alpha = alpha), X_train, y_train).mean() for alpha in alphas]

cv_ridge = pd.Series(cv_ridge, index = alphas)
cv_ridge.plot(title = "Validation", figsize=(5,5))
plt.xlabel("alpha")
plt.ylabel("rmse")

model_ridge = Ridge(alpha = cv_ridge.min()).fit(X_train, y_train)

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % model_ridge.score(X_test, y_test))

coef = pd.Series(model_ridge.coef_, index = df_autos.iloc[:,1:].columns)

imp_coef = pd.concat([coef.sort_values()])

print(imp_coef)

print(np.sqrt(np.mean((y_test - model_ridge.predict(X_test))**2)))
print("rmse on validation set", cv_rmse(model_ridge, X_test, y_test).mean())

# Last version results:
# 0.639334838313
# rmse on validation set 0.639555431574


#### Predicting samples

In [ ]:
vw1 = calc_values(model_ridge, 2016, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 105, 10000, 'nein')
bmw  = calc_values(model_ridge, 2016, 'bmw', '3er', 'limousine', 'manuell', 'diesel', 150, 10000, 'nein')
fiat  = calc_values(model_ridge, 2016, 'fiat', 'punto', 'kleinwagen', 'manuell', 'benzin', 60, 10000, 'nein')
vw2 = calc_values(model_ridge, 2016, 'volkswagen', 'tiguan', 'suv', 'manuell', 'diesel', 130, 10000, 'nein')

print(vw1)
print(bmw)
print(fiat)
print(vw2)

values_ridge  = [calc_values(model_ridge, y, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 120, km_year * (2017-y), 'nein') for y in r]
values_ridge2  = [calc_values(model_ridge, y, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 120, km_year * 2 * (2017-y), 'nein') for y in r]
values_ridge3  = [calc_values(model_ridge, y, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 120, km_year * 3 * (2017-y), 'nein') for y in r]
plt.plot(r, values_ridge, label='ridge')
plt.plot(r, values_ridge2, label='ridge 20000km_x_y')
plt.plot(r, values_ridge3, label='ridge 30000km_x_y')

#plt.grid()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


## Random forests

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators = 50)
forest.fit(X_train, y_train)

In [ ]:
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % forest.score(X_test, y_test))

print(np.sqrt(np.mean((y_test - forest.predict(X_test))**2)))
print("rmse on validation set", cv_rmse(forest, X_test, y_test).mean())

# Last version results
# Variance score: 0.88
# 0.391895988852
# rmse on validation set 0.426248010639


#### Predicting samples

In [ ]:
vw1 = calc_values(forest, 2016, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 105, 10000, 'nein')
bmw  = calc_values(forest, 2016, 'bmw', '3er', 'limousine', 'manuell', 'diesel', 150, 10000, 'nein')
fiat  = calc_values(forest, 2016, 'fiat', 'punto', 'kleinwagen', 'manuell', 'benzin', 60, 10000, 'nein')
vw2 = calc_values(forest, 2016, 'volkswagen', 'tiguan', 'suv', 'manuell', 'diesel', 130, 10000, 'nein')

print(vw1)
print(bmw)
print(fiat)
print(vw2)

#values_forest = [calc_values(forest     , y, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 120, km_year * (2017-y), 'nein') for y in r]
#values_forest2 = [calc_values(forest     , y, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 120, km_year* 2 * (2017-y), 'nein') for y in r]
#values_forest3 = [calc_values(forest     , y, 'volkswagen', 'touran', 'bus', 'manuell', 'diesel', 120, km_year* 3 * (2017-y), 'nein') for y in r]

values_forest  = [calc_values(forest, y, 'audi', 'a3', 'coupe', 'manuell', 'diesel', 140, km_year * (2017-y), 'nein') for y in r]
values_forest4 = [calc_values(forest, y, 'volkswagen', 'golf', 'coupe', 'manuell', 'diesel', 140, km_year * (2017-y), 'nein') for y in r]
values_forest5 = [calc_values(forest, y, 'seat', 'ibiza', 'coupe', 'manuell', 'diesel', 140, km_year * (2017-y), 'nein') for y in r]
values_forest6 = [calc_values(forest, y, 'dacia', 'logan', 'coupe', 'manuell', 'diesel', 140, km_year * (2017-y), 'nein') for y in r]


plt.plot(r, values_forest, label='audi')
plt.plot(r, values_forest4, label='vw')
plt.plot(r, values_forest5, label='seat')
plt.plot(r, values_forest6, label='dacia')

#plt.grid()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


## Testing XGBoost

In [ ]:
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

clf = XGBRegressor()
clf.fit(X_train, y_train)


In [ ]:
xgb.plot_importance(clf)

#f0 = year
#f1 = powerPS
#f2 = km
#f3 = month
#f4 = namelen
#f5 = gearbox
#f6 = notRepairedDamage
#f7 = model
#f8 = brand
#f9 = fuelType
#f10 = vehicleType

# make predictions for test data
y_pred = clf.predict(X_test)
print(np.sqrt(np.mean((y_test - y_pred)**2)))

# Last version results
# 0.433305159267


#### Predicting samples

In [ ]:
values_clf  = [calc_values(clf, y, 'audi', 'a3', 'coupe', 'manuell', 'diesel', 140, km_year * (2017-y), 'nein') for y in r]
values_clf2 = [calc_values(clf, y, 'audi', 'a3', 'coupe', 'manuell', 'diesel', 105, km_year * (2017-y), 'nein') for y in r]
values_clf3 = [calc_values(clf, y, 'audi', 'a3', 'coupe', 'manuell', 'diesel', 200, km_year * (2017-y), 'nein') for y in r]
values_clf  = [calc_values(clf, y, 'audi', 'a3', 'coupe', 'manuell', 'diesel', 140, km_year * (2017-y), 'nein') for y in r]
values_clf4 = [calc_values(clf, y, 'volkswagen', 'golf', 'coupe', 'manuell', 'diesel', 140, km_year * (2017-y), 'nein') for y in r]
values_clf5 = [calc_values(clf, y, 'seat', 'ibiza', 'coupe', 'manuell', 'diesel', 140, km_year * (2017-y), 'nein') for y in r]
values_clf6 = [calc_values(clf, y, 'dacia', 'logan', 'coupe', 'manuell', 'diesel', 140, km_year * (2017-y), 'nein') for y in r]
#plt.plot(r, values_clf2, label='audi 105cv')
#plt.plot(r, values_clf3, label='audi 200cv')
plt.plot(r, values_clf , label='audi 140cv')
plt.plot(r, values_clf4, label='golf ')
plt.plot(r, values_clf5, label='ibiza ')
plt.plot(r, values_clf6, label='dacia ')

#plt.grid()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


# Conclusions

I've tried to play with as much stuff as I could with this dataset in order to understand the very basic topics about:

* data interpretation and selection
* feature selection and labeling
* data visualization
* very rough ML algorithms application

There's very much to improve both in how I managed all these steps and in the different outcomes of the predictions on the sell price. I'll experiment a bit more in the next few days, then I'll move on another dataset to learn more.
